In [13]:
import wandb
import pandas as pd

In [15]:
run = wandb.init(project = "eda",group = "eda",job_type = "eda")

In [3]:
input_artifact = 'shubha_manikarnike/nyc_airbnb/sample.csv:v0'
artifact = run.use_artifact(input_artifact).file()

In [5]:
df = pd.read_csv(artifact)

In [6]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,3,0,NaN,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,3,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,3,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,2,8,2019-06-23,0.52,2,8


In [16]:
wandb.log({"table": df})

In [17]:
def make_split_artifact(run, raw_data_table, train_rows, val_rows, test_rows):
    """
    Creates a w&b artifact that contains a singular reference table (aka a ForeignIndex table).
    The ForeignIndex table has a single column that we are naming 'source'.
    It contains references to the original table (raw_data_table) for each of the splits.
    Arguments:
        run (wandb run) returned from wandb.init()
        raw_data_table (wandb Table) that contains your original tabular data
        train_rows (list of ints) indices that reference the training rows in the raw_data_table
        val_rows (list of ints) indices that reference the validation rows in the raw_data_table
        test_rows (list of ints) indices that reference the test rows in the raw_data_table
    """
    split_artifact = wandb.Artifact(
        'data-splits', type='dataset',
        description='Train, validation, test dataset splits')
    # Our data split artifact will only store index references to the original dataset table to save space
    data_table_pointer = raw_data_table.get_index() # ForeignIndex automatically references the source table
    split_artifact.add(wandb.Table(
        columns=['source'],
        data=[[data_table_pointer[i]] for i in train_rows]), 'train-data')
    split_artifact.add(wandb.Table(
        columns=['source'],
        data=[[data_table_pointer[i]] for i in val_rows]), 'val-data')
    split_artifact.add(wandb.Table(
        columns=['source'],
        data=[[data_table_pointer[i]] for i in test_rows]), 'test-data')
    run.log_artifact(split_artifact)


In [18]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df, test_size=0.2, random_state=42)

In [19]:
train,val = train_test_split(train, test_size=0.2, random_state=42)

In [20]:
train.shape, val.shape, test.shape

((12800, 16), (3200, 16), (4000, 16))

In [24]:

my_table = run.use_artifact("shubha_manikarnike/eda/run-c0xeitw2-table:v0")

In [26]:
run.finish()

In [27]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [28]:
# Download a simple dataset
iris = load_iris()
# Load it into a dataframe
iris_dataframe = pd.DataFrame(data=np.c_[iris['data'], iris['target']],
                     columns=iris['feature_names'] + ['target'])

In [29]:
iris_dataframe

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [30]:
# Start a W&B run to log data
wandb.init(project="Tables-Quickstart")
# Log the dataframe to visualize
wandb.log({"iris": iris_dataframe})
# Finish the run (useful in notebooks)
wandb.finish()